In [ ]:
# pip install opencv-contrib-python --user
# pip install pywhatkit
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join
import os
import smtplib as s
import pywhatkit
import subprocess
import time

In [ ]:
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_extractor(img):
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    
    if faces is ():
        return None
    
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h, x:x+w]
    return cropped_face

In [ ]:
cap = cv2.VideoCapture(0)
count = 0

In [ ]:
while True:

    ret, frame = cap.read()
    if face_extractor(frame) is not None:
        count += 1
        face = cv2.resize(face_extractor(frame), (200, 200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        file_name_path = 'C:/Users/Mukul/Desktop/summertraining/CV/faces_cv/2/' + str(count) + '.jpg'
        cv2.imwrite(file_name_path, face)

        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Face', face)
    else:
        print("Face not found")
        pass

    if cv2.waitKey(1) == 13 or count == 100: 
        break
        
cap.release()
cv2.destroyAllWindows()      
print("Done!")

In [ ]:
count = 1
model = []
while True:
    data_path = 'C:/Users/Mukul/Desktop/summertraining/CV/faces_cv/'+str(count)+'/'
    onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]
    Training_Data, Labels = [], []

    for i, files in enumerate(onlyfiles):
        image_path = data_path + onlyfiles[i]
        images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        Training_Data.append(np.asarray(images, dtype=np.uint8))
        Labels.append(i)
    
    Labels = np.asarray(Labels, dtype=np.int32)

    model.append(cv2.face_LBPHFaceRecognizer.create())

    model[count-1].train(np.asarray(Training_Data), np.asarray(Labels))
    print("Model_"+str(count)+ " trained sucessefully")
    count = count +1
    if(count > 2):
        break

In [ ]:
Name = ["Hritik","Criss"]

In [ ]:
def get_confidence(results):
    confidence=[]
    for i in range(0,2):
        if results[i][1] < 500:
            confidence.append(int( 100 * (1 - (results[i][1])/400) )) 
    
    for i in range(0,2):
        if confidence[0]>88:
            display_string = str(confidence[i]) + '% Confident it is '+Name[0]
            cv2.putText(image, display_string, (100, 400), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
            cv2.putText(image, "Hey "+Name[0], (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Recognition', image )
            time.sleep(5)
            send_email()
            do_whatsapp()
            break
        elif confidence[1]>82:
            display_string = str(confidence[i]) + '% Confident it is '+Name[1]
            cv2.putText(image, display_string, (100, 400), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
            cv2.putText(image, "Hey "+Name[1], (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Recognition', image )
            time.sleep(5)
            create_aws()
            time.sleep(3)
            break
    else:
        cv2.imshow('Face Recognition', image )

In [ ]:
def send_email():
    file = open("password.txt")
    password = file.read()
    e = s.SMTP("smtp.gmail.com", 587)
    e.starttls()
    e.login("mukulpramanik48@gmail.com", password)
    subject = "OpenCV email"
    body = "This mail is for testing purpose."
    message = "Subject:{}\n\n{}".format(subject,body)
    email_list = ["mukulpramanik329@outlook.com"]
    e.sendmail("mukulpramanik48@gmail.com", email_list, message)
    print("Email send successfully...")
    e.quit()

In [ ]:
def do_whatsapp():
    file = open("phone_no.txt")
    phone_no = file.read()
    pywhatkit.sendwhatmsg_instantly(phone_no,"hey")
    print("Whatsapp message send successfully...")

In [ ]:
def create_aws():
    awsec2 = open("awsec2_cli.txt")
    instance_id = subprocess.getoutput(awsec2.read())
    print("Instance created\nInstance ID: ", instance_id)
    awsebs = open("awsebs_cli.txt")
    volume_id = subprocess.getoutput(awsebs.read())
    print("Volume created\nVolume ID: ", volume_id)
    time.sleep(20)
    subprocess.getoutput("aws ec2 attach-volume --volume-id {} --instance-id {} --dev /dev/sdf".format(volume_id, instance_id))
    print("Volume successfully attached....")

In [ ]:
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_detector(img, size=0.5):
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []
    
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi


cap = cv2.VideoCapture(0)

while True:

    ret, frame = cap.read()
    
    image, face = face_detector(frame)
    
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        results = []
        
        for i in range(0,2):
            if i == 0:
                results.append(model[i].predict(face))
            elif i == 1:
                results.append(model[i].predict(face))
        get_confidence(results)

    except:
        cv2.putText(image, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face Recognition', image )
        pass
        
    if cv2.waitKey(1) == 13:
        break
        
cap.release()
cv2.destroyAllWindows()